# HVG - Split

In [1]:
import scanpy as sc
import numpy as np
import sys
sys.path.append("../src_code") 
from preprocess_data import _hvg_batch

print('before load data')
path = "../data/"



before load data


In [2]:
adata = sc.read_h5ad(path+'Hpoly_salmonella.h5ad')
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [3]:
condition_key = 'condition'
label_key = 'cell_label'
adata.obs[label_key].value_counts()

TA.Early                 1970
Stem                     1857
Enterocyte.Progenitor    1822
Enterocyte               1458
TA                       1188
Goblet                    758
Tuft                      409
Endocrine                 380
Name: cell_label, dtype: int64

In [4]:
# adata = adata[adata.obs[label_key].isin(['CD14 Mono',
#                            'CD4 Naive T',
#                            'CD4 Memory T',
#                            'CD16 Mono',
#                            'B',
#                            'CD8 T',
#                            'T activated',
#                            'NK'])]

In [5]:
adata.obs[label_key].value_counts()

TA.Early                 1970
Stem                     1857
Enterocyte.Progenitor    1822
Enterocyte               1458
TA                       1188
Goblet                    758
Tuft                      409
Endocrine                 380
Name: cell_label, dtype: int64

In [6]:
adata.obs[condition_key] = adata.obs[condition_key].astype('category')

In [7]:
adata_hvg = _hvg_batch(adata,batch_key=condition_key, target_genes=2000, adataOut=True)

/Users/ekarimi/opt/anaconda3/envs/SCDI/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Using 534 HVGs from full intersect set
Using 376 HVGs from n_batch-1 set
Using 909 HVGs from n_batch-2 set
Using 181 HVGs from n_batch-3 set
Using 2000 HVGs


In [8]:
adata.obs[condition_key].value_counts()

Control        3240
Hpoly.Day10    2711
Hpoly.Day3     2121
Salmonella     1770
Name: condition, dtype: int64

In [9]:
condition_1= 'Control'
condition_2= 'Hpoly.Day10'

In [10]:
adata1 = adata_hvg[adata_hvg.obs[condition_key]=='Control']

In [11]:
adata2 = adata_hvg[adata_hvg.obs[condition_key]=='Hpoly.Day10']

In [12]:
for i in list(adata1.obs[label_key].values.categories):
    adata1.obs[label_key] = adata1.obs[label_key].cat.rename_categories({i: "_".join(i.split())+'_1'})

Trying to set attribute `.obs` of view, copying.


In [13]:
adata1.obs[label_key].values.categories

Index(['Endocrine_1', 'Enterocyte_1', 'Enterocyte.Progenitor_1', 'Goblet_1',
       'Stem_1', 'TA_1', 'TA.Early_1', 'Tuft_1'],
      dtype='object')

In [14]:
for i in list(adata2.obs[label_key].values.categories):
    adata2.obs[label_key] = adata2.obs[label_key].cat.rename_categories({i: "_".join(i.split())+'_2'})

Trying to set attribute `.obs` of view, copying.


In [15]:
adata2.obs[label_key].values.categories

Index(['Endocrine_2', 'Enterocyte_2', 'Enterocyte.Progenitor_2', 'Goblet_2',
       'Stem_2', 'TA_2', 'TA.Early_2', 'Tuft_2'],
      dtype='object')

In [16]:
new_adata = adata1.concatenate(adata2)

In [17]:
new_adata.obs[label_key].value_counts()

TA.Early_1                 792
Stem_1                     670
Stem_2                     592
Enterocyte.Progenitor_2    586
Enterocyte.Progenitor_1    545
TA.Early_2                 436
Enterocyte_1               424
TA_1                       421
TA_2                       353
Goblet_2                   317
Tuft_2                     217
Goblet_1                   216
Enterocyte_2               128
Endocrine_1                112
Endocrine_2                 82
Tuft_1                      60
Name: cell_label, dtype: int64

In [18]:
new_adata.write_h5ad(path+'adata_hvg.h5ad')

... storing 'barcode' as categorical
... storing 'condition' as categorical
... storing 'cell_label' as categorical


# Train Test Split

In [19]:
from preprocess_data import train_test_split

In [20]:
train_adata, test_adata = train_test_split(adata_hvg[adata_hvg.obs[condition_key]==condition_1])

In [21]:
train_adata.write_h5ad(path+'train_1.h5ad')

In [22]:
test_adata.write_h5ad(path+'test_1.h5ad')

In [23]:
train_adata, test_adata = train_test_split(adata_hvg[adata_hvg.obs[condition_key]==condition_2])

In [24]:
train_adata.write_h5ad(path+'train_2.h5ad')

In [25]:
test_adata.write_h5ad(path+'test_2.h5ad')

#### Test the code is working or not

In [25]:
# from all_code import MLP, Trainer

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')


s_x, s_y = 1, 2
num_classes = np.unique(train_adata.obs[label_key]).shape[0]

model = MLP(train_adata.n_vars, num_classes=num_classes, layer_sizes=[512, 128, 256], dr_rate=0.0)
model.to(device)

Using device: cpu


MLP(
  (FC): Sequential(
    (L0): Linear(in_features=4000, out_features=512, bias=False)
    (B0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A0): ReLU()
    (D0): Dropout(p=0.0, inplace=False)
    (L1): Linear(in_features=512, out_features=128, bias=False)
    (B1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A1): ReLU()
    (D1): Dropout(p=0.0, inplace=False)
    (L2): Linear(in_features=128, out_features=256, bias=False)
    (B2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A2): ReLU()
    (D2): Dropout(p=0.0, inplace=False)
    (output): Linear(in_features=256, out_features=8, bias=True)
  )
)

In [27]:
c = train_adata.copy()

In [28]:
c.X = c.X.A

In [29]:
trainer = Trainer(model, c, label_key=label_key)
trainer.train_nll( 150, 32, weight_decay=0)

Epoch 00/150 Batch 0143/143, Loss:    0.4924
Epoch 01/150 Batch 0143/143, Loss:    0.1741
Epoch 02/150 Batch 0143/143, Loss:    0.1045
Epoch 03/150 Batch 0143/143, Loss:    0.0796
Epoch 04/150 Batch 0143/143, Loss:    0.0285
Epoch 05/150 Batch 0143/143, Loss:    0.1066
Epoch 05, Loss_valid:    6.5680, acc_valid:    0.9252


KeyboardInterrupt: 

In [30]:

#train report
from sklearn.metrics import classification_report

print("******************")
print("train on facs")
y_pred = model.label_encoder.inverse_transform(model.predict(trainer.train_adata.X))
print(classification_report(np.array(trainer.train_adata.obs[label_key]).reshape(-1,1), y_pred))

print("******************")
print("validation")
y_pred = model.label_encoder.inverse_transform(model.predict(trainer.validation_adata.X))
print(classification_report(np.array(trainer.validation_adata.obs[label_key]).reshape(-1,1), y_pred))


******************
train on facs
              precision    recall  f1-score   support

           B       1.00      1.00      1.00       365
   CD14 Mono       1.00      1.00      1.00      1475
   CD16 Mono       0.99      1.00      0.99       369
CD4 Memory T       1.00      1.00      1.00       626
 CD4 Naive T       1.00      1.00      1.00      1038
       CD8 T       1.00      1.00      1.00       314
          NK       1.00      1.00      1.00       213
 T activated       0.99      1.00      1.00       224

    accuracy                           1.00      4624
   macro avg       1.00      1.00      1.00      4624
weighted avg       1.00      1.00      1.00      4624

******************
validation
              precision    recall  f1-score   support

           B       0.99      1.00      0.99        85
   CD14 Mono       0.99      1.00      0.99       263
   CD16 Mono       0.98      0.98      0.98        57
CD4 Memory T       0.83      0.89      0.86       108
 CD4 Naive T   

In [31]:
ood = adata_hvg[adata_hvg.obs[condition_key]=='CTRL']

/home/karimiara.ce.sharif/.conda/envs/scdi/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [32]:
print("validation")
d = ood.copy()
d.X =d.X.A
y_pred = model.label_encoder.inverse_transform(model.predict(d.X))
print(classification_report(np.array(d.obs[label_key]).reshape(-1,1), y_pred))


validation
              precision    recall  f1-score   support

           B       0.98      0.98      0.98       407
   CD14 Mono       1.00      0.97      0.98      2215
   CD16 Mono       0.93      0.96      0.95       507
CD4 Memory T       0.76      0.92      0.83       859
 CD4 Naive T       0.95      0.81      0.88       978
       CD8 T       0.86      0.69      0.77       352
          NK       0.95      0.93      0.94       298
 T activated       0.72      0.97      0.83       300

    accuracy                           0.92      5916
   macro avg       0.90      0.90      0.89      5916
weighted avg       0.92      0.92      0.92      5916

